In [29]:
import matplotlib.pyplot as plt
import pandas as pd
import requests
from io import StringIO
import altair as alt


url = "https://raw.githubusercontent.com/melaniewalsh/responsible-datasets-in-context/main/datasets/top-500-novels/library_top_500.csv"


response = requests.get(url, verify=False)
csv_data = StringIO(response.text)
df = pd.read_csv(csv_data, sep=',', header=0, low_memory=False)
df

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,top_500_rank,title,author,pub_year,orig_lang,genre,author_birth,author_death,author_gender,author_primary_lang,...,gr_num_ratings,gr_num_reviews,gr_avg_rating_rank,gr_num_ratings_rank,oclc_owi,author_viaf,gr_url,wiki_url,pg_eng_url,pg_orig_url
0,1,Don Quixote,Miguel de Cervantes,1605,Spanish,action,1547,1616,male,spa,...,"269,435","12,053",318,211,1.810748e+09,17220427,https://www.goodreads.com/book/show/3836.Don_Q...,https://en.wikipedia.org/wiki/Don_Quixote,https://www.gutenberg.org/cache/epub/996/pg996...,https://www.gutenberg.org/cache/epub/2000/pg20...
1,2,Alice's Adventures in Wonderland,Lewis Carroll,1865,English,fantasy,1832,1898,male,eng,...,"561,016","15,380",172,133,1.156132e+10,66462036,https://www.goodreads.com/book/show/24213.Alic...,https://en.wikipedia.org/wiki/Alice%27s_Advent...,https://www.gutenberg.org/cache/epub/11/pg11.txt,NaN
2,3,The Adventures of Huckleberry Finn,Mark Twain,1884,English,action,1835,1910,male,eng,...,"1,262,480","19,440",373,68,3.373178e+09,50566653,https://www.goodreads.com/book/show/2956.The_A...,https://en.wikipedia.org/wiki/Adventures_of_Hu...,https://www.gutenberg.org/cache/epub/76/pg76.txt,NaN
3,4,The Adventures of Tom Sawyer,Mark Twain,1876,English,action,1835,1910,male,eng,...,"931,898","13,603",301,88,3.373178e+09,50566653,https://www.goodreads.com/book/show/24583.The_...,https://en.wikipedia.org/wiki/The_Adventures_o...,https://www.gutenberg.org/cache/epub/74/pg74.txt,NaN
4,5,Treasure Island,Robert Louis Stevenson,1883,English,action,1850,1894,male,eng,...,"486,155","16,307",368,145,3.434000e+03,95207986,https://www.goodreads.com/book/show/295.Treasu...,https://en.wikipedia.org/wiki/Treasure_Island,https://www.gutenberg.org/cache/epub/120/pg120...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,Stranger in a Strange Land,Robert A. Heinlein,1961,English,scifi,1907,1988,male,eng,...,"311,859","9,961",310,190,7.894120e+05,12309757,NaN,https://en.wikipedia.org/wiki/Stranger_in_a_St...,NA_not-pub-domain,NaN
496,497,Vision in White,Nora Roberts,2009,English,romance,1965,ALIVE,female,eng,...,"138,445","4,652",128,277,1.559638e+08,66448023,NaN,https://en.wikipedia.org/wiki/Vision_in_White,NA_not-pub-domain,NaN
497,498,The Whipping Boy,Sid Fleischman,1986,English,action,1920,2010,male,eng,...,"27,444","1,623",476,445,4.415520e+08,66438084,NaN,https://en.wikipedia.org/wiki/The_Whipping_Boy,NA_not-pub-domain,NaN
498,499,Room,Emma Donoghue,2010,English,na,1969,ALIVE,female,eng,...,"801,989","50,594",171,101,4.859780e+08,39539889,NaN,https://en.wikipedia.org/wiki/Room_(novel),NA_not-pub-domain,NaN


From this dataset, we will take a look at gender bias in ranking. We will specifically take a look at the top 50 ranked books and the gender distribution of those, then look at the bottom 50 and look at the gender distribution of those. From this we will create graphics to show us the differnce or similarities so we can draw our conclusion. 

Now, we will cut down the datset to only show us the gender of the author and the ranking of the book

In [30]:
df_selected = df[['author_gender', 'top_500_rank']]
df_selected

,author_gender,top_500_rank
0,male,1
1,male,2
2,male,3
3,male,4
4,male,5
...,...,...
495,male,496
496,female,497
497,male,498
498,female,499


Now we will make it show us only top 50 rankings and bottom 50 rankings. 

In [31]:
top50 = df_selected[(df_selected['top_500_rank'] >= 1) & (df_selected['top_500_rank'] <= 50)].reset_index(drop=True)
bot50 = df_selected[(df_selected['top_500_rank'] >= 451) & (df_selected['top_500_rank'] <= 500)].reset_index(drop=True)

In [32]:
print(top50.to_string(index=False))

author_gender  top_500_rank
         male             1
         male             2
         male             3
         male             4
         male             5
       female             6
       female             7
       female             8
         male             9
         male            10
         male            11
         male            12
         male            13
         male            14
         male            15
       female            16
         male            17
         male            18
       female            19
         male            20
       female            21
         male            22
         male            23
         male            24
         male            25
         male            26
         male            27
         male            28
       female            29
         male            30
         male            31
       female            32
         male            33
         male            34
         male       

In [33]:
print(bot50.to_string(index=False))

author_gender  top_500_rank
       female           451
       female           452
         male           453
         male           454
         male           455
         male           456
         male           457
         male           458
         male           459
       female           460
         male           461
       female           462
         male           463
         male           464
       female           465
         male           466
         male           467
       female           468
         male           469
       female           470
         male           471
         male           472
         male           473
         male           474
         male           475
         male           476
         male           477
         male           478
       female           479
         male           480
       female           481
         male           482
         male           483
         male           484
         male       

Now i will count how many males there are to females in top50 and then the same for the bottom 50

In [34]:
top50_counts = top50['author_gender'].value_counts()
bot50_counts = bot50['author_gender'].value_counts()

In [35]:
top50_counts

male      39
female    11
Name: author_gender, dtype: int64

In [36]:
bot50_counts

male      37
female    13
Name: author_gender, dtype: int64

From this information, we can see that there is a 39/11 male to female ratio in the top 50 of publishers and 37/13 male to female in the bottom 50 of publishers. 
Now we will make some graphs to look at it visually. 

In [39]:
bar_top50 = alt.Chart(top50_counts).mark_bar().encode(
    x=alt.X('author_gender:N', title='Gender'),
    y=alt.Y('count:Q', title='Count'),
    color='author_gender:N'
).properties(
    title='Gender Distribution in Top 50 Rankings'
)
bar_top50

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/altair/vegalite/v5/api.py:290: UserWarning: data of type <class 'pandas.core.series.Series'> not recognized
  warnings.warn(f"data of type {type(data)} not recognized", stacklevel=1)


SchemaValidationError: '[39, 11]' is an invalid value for `data`. Valid values are of type 'object' or 'null'.

alt.Chart(...)